# Monitor All Simulations

This programs allows you to do almost everything in order to monitor your simulations. It is a graphical interface that is very simple to use. When you execute it, it fetches all the simulation folders and for each one it computes the order parameter if needed and then retrieves 4 things : the *order-parameter*, the *sign*, the *oxygen and copper occupations* as functions of **iteration**. This operation is done in the background in another thread. In the meantime, as soon as the information is downloaded, the program displays 3 graphs for the first folder : *order parameter*, *sign* and *total occupation* as functions of **iteration**. You can then choose some options : 
* **Cancel Simulation** cancels the ongoing simulation.

* **Copy to computer** prepares the folder for copy to the computer (by deleting useless files like config files or scripts and logfiles) and then uses *scp* to copy the files in your AllData/transfered folder. This operation is done in another thread so you can keep on with your life and the other folders. When the transfer is finished, it prints the folder name in the file AllData/transfer.done and removes the remote folder. Be careful not to stop the kernel before the file transfer is done as it may result on deleting the remote folder prematurely and using losing your simulation results.


* **Run all occupations** runs Full Green's function computation (GFULL) for all the iterations on which this was not already called.


* **Delete last iteration** deletes the last iteration that was done. This is not used a lot but can be useful in case somehting werid happened (the simulations stops during the self-consistency relation or somehting like that).


* **Resume simulation** uses the two inputs on the front to resume the simulation. The two inputs should be integers. The first input is the max iteration you want to stop at (if it is -1, there is no limit on iterations, only on computing time). The second input should be the time in days you want to run the simulation.

* **+1 day** adds one day of simulation (equivalent of the Resume Simulation button with inputs -1 and 1)

* **+2 days** adds two days of simulation (equivalent of the Resume Simulation button with inputs -1 and 2)


When you are finished and by pressing the Next button, you can have the same options and display for the second folder when the data is loaded. It goes on until every folder is done. If there is an error for a folder it is generally not displayed. Be careful about that.

Multiple red lines may also appear in the output panel. Those often come from matplotlib operations within the graphical interface (google says it's normal and one **shouldn't** worry). Some red text may also come from ssh requests done within the code.

The order parameter is computed using the `compute_order_parameter` function in the `scripts/action.py` file and saved to a file in the simulation folder. This program then retrieves the file.

If the file is 1 dimensional, the data is just plotted normally

If the file is more than 1 dimensional, the 1st component along the 1st axis ([:,0]) is plotted normally. The other components ([:,i]) are normalized so that the data can be monitored on one plot easily. You can then only monitor a relative convergence.

In [1]:
import sys
import os
from fabric.connection import Connection
from AnalysisUtilities import distant_consts as CONSTS
from importlib import reload
all_datafolder = os.path.join(CONSTS.distant_main_dir,"ComputedData")
all_folder_names = []
with Connection(CONSTS.ssh_address) as c:
    result = c.run("cd " + all_datafolder + ";ls",hide=True)
    folders = result.stdout.split()
    for f in folders:
        all_folder_names.append(f)
       
print("Connected to " + CONSTS.ssh_address)
#Launch the data transfer
all_folder_data = []
from AnalysisUtilities import distant_utils as UTILS
from threading import Thread
from tkinter import messagebox
Thread(target=UTILS.get_all_folder_data,args=(all_folder_data,all_folder_names,c)).start() 

#The window
from AnalysisUtilities import custom_window as cw
reload(cw)
mean_over = 15
for i,folder_name in enumerate(all_folder_names):
    while(i >= len(all_folder_data)):
        pass
    if all_folder_data[i] != 0:
        cw.Process_window(all_folder_data[i],mean_over,c) 
    else:
        print("On a rien pour" + folder_name)
        messagebox.showwarning("No enough data in folder", "For this simulation folder (" + folder_name + "), there was not enough data (usually no simulation was done). Please use the New Simulation script to resume the simulation.")

Connected to kowalski@cedar.computecanada.ca
Finished
